<h1>Decision Tree</h1>

In [21]:
library(mlr)
source('../utils.r')

set.seed(42)

folder_name = '../../raw_data' 
file_name   = 'german.csv'

<h2>1. Dataprep</h2>

In [22]:
data = read.csv(file=sprintf('%s/%s',folder_name,file_name))

In [23]:
cat(sprintf('NRow: %d\nNCol: %d',nrow(data), ncol(data)))
head(data)

NRow: 1000
NCol: 22

X,V1,V2,V3,V4,V5,V6,V7,V8,V9,⋯,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21
1,A11,6,A34,A43,1169,A65,A75,4,A93,⋯,A121,67,A143,A152,2,A173,good,A192,A201,1
2,A12,48,A32,A43,5951,A61,A73,2,A92,⋯,A121,22,A143,A152,1,A173,good,A191,A201,2
3,A14,12,A34,A46,2096,A61,A74,2,A93,⋯,A121,49,A143,A152,1,A172,bad,A191,A201,1
4,A11,42,A32,A42,7882,A61,A74,2,A93,⋯,A122,45,A143,A153,1,A173,bad,A191,A201,1
5,A11,24,A33,A40,4870,A61,A73,3,A93,⋯,A124,53,A143,A153,2,A173,bad,A191,A201,2
6,A14,36,A32,A46,9055,A65,A73,2,A93,⋯,A124,35,A143,A153,1,A172,bad,A192,A201,1


<p style='color: red'> ATENTION: </p>Specifically in R, in classification problem, target must be set as Factor.

In [24]:
data$V21 = as.factor(data$V21)

MLR works only with features and target, this means that others columns must be dorped.

In [25]:
drops = c()
data  = data[ , !(names(data) %in% drops)]
cat(sprintf('NRow: %d\nNCol: %d',nrow(data), ncol(data)))

NRow: 1000
NCol: 22

<h2>2. Modeling</h2>

Function
```R
makeLearner(cl, id = cl, predict.type = "response", predict.threshold = NULL, 
            fix.factors.prediction = FALSE, ..., par.vals = list(), config = list())
```
Param.:

* cl: [character(1)] Class of learner. By convention, all classification learners start with “classif.”. A list of all integrated learners is available on the learners help page < https://mlr-org.github.io/mlr-tutorial/release/html/integrated_learners/ >.
* predict: [character(1)] “response” (= labels) or “prob” (= probabilities and labels by selecting the ones with maximal probability). Default is “response”.
* par.vals: [list] Optional list of named (hyper)parameters. The arguments in ... take precedence over values in this list. We strongly encourage you to use one or the other to pass (hyper)parameters to the learner but not both.

Doc.: https://www.rdocumentation.org/packages/mlr/versions/2.10/topics/makeLearner <br>
Rpart Parameters: https://www.rdocumentation.org/packages/rpart/versions/4.1-11/topics/rpart.control

In [60]:
cl = "classif.ranger"

List all parameters that can be used in this classifier.
The value must be set in par.vals parameter.

In [61]:
getParamSet(cl)

                                      Type  len    Def
num.trees                          integer    -    500
mtry                               integer    -      -
mtry.perc                          numeric    -      -
min.node.size                      integer    -      -
replace                            logical    -   TRUE
sample.fraction                    numeric    -      -
split.select.weights         numericvector <NA>      -
always.split.variables             untyped    -      -
respect.unordered.factors         discrete    - ignore
importance                        discrete    -   none
write.forest                       logical    -   TRUE
scale.permutation.importance       logical    -  FALSE
num.threads                        integer    -      -
save.memory                        logical    -  FALSE
verbose                            logical    -   TRUE
seed                               integer    -      -
splitrule                         discrete    -   gini
num.random

In [62]:
learner = makeLearner(cl = cl
                     , predict.type = "prob"
                     , par.vals = list()
                     )

Function
```R
makeClassifTask(id = deparse(substitute(data)), data, target, weights = NULL, blocking = NULL, 
                positive = NA_character_, fixup.data = "warn", check.data = TRUE)
```
Param.:

* data: [data.frame] A data frame containing the features and target variable(s).
* target: [character(1)] Name of the target variable.
* positive: [character(1)] Positive class for binary classification (otherwise ignored and set to NA). Default is the first factor level of the target attribute.
* fixup.data: [character(1)] Should some basic cleaning up of data be performed? Currently this means removing empty factor levels for the columns. Possible coices are: “no” = Don't do it. “warn” = Do it but warn about it. “quiet” = Do it but keep silent. Default is “warn”.

Doc.: https://www.rdocumentation.org/packages/mlr/versions/2.10/topics/makeLearner

In [63]:
task = makeClassifTask( data = data
                      , target = 'V21'
                      , positive = '2'
                      , fixup.data = 'no'
                      )

Function:
```R
makeResampleDesc(method, predict = "test", ..., stratify = FALSE, stratify.cols = NULL)
```
Param.:

* method: [character(1)] “CV” for cross-validation, “LOO” for leave-one-out, “RepCV” for repeated cross-validation, “Bootstrap” for out-of-bag bootstrap, “Subsample” for subsampling, “Holdout” for holdout.
* predict: What to predict during resampling: “train”, “test” or “both” sets. Default is “test”.
* ... : [any] Further parameters for strategies.
    * iters [integer(1)] Number of iterations, for “CV”, “Subsample” and “Boostrap”.
    * split [numeric(1)] Proportion of training cases for “Holdout” and “Subsample” between 0 and 1. Default is 2/3.
    * reps [integer(1)] Repeats for “RepCV”. Here iters = folds * reps. Default is 10.
    * folds [integer(1)] Folds in the repeated CV for RepCV. Here iters = folds * reps. Default is 10.

Doc.: https://www.rdocumentation.org/packages/mlr/versions/2.10/topics/makeResampleDesc

In [64]:
resample = makeResampleDesc(
    method = "CV",
    iters = 10,
    predict = 'both',
    stratify = FALSE
)

List of performance measures:

Doc.: http://mlr-org.github.io/mlr-tutorial/release/html/measures/

In [65]:
measures = list(mmce #MMCE 
               ,acc  #acuracia
               ,f1   #f1
               ,ppv  #precision
               ,tpr  #recall
               ,auc  #AUC
               ,gini #Gini
               #,timetrain #tempo execucao
               )

Function:
```R
resample(learner, task, resampling, measures, weights = NULL, models = FALSE, extract, 
         keep.pred = TRUE, ..., show.info = getMlrOption("show.info"))
```
Param.:

* learner: [Learner] The learner.
* task: [Task] The task.
* resampling: [ResampleInstance] Resampling strategy.
* measures: [Measure | list of Measure] Performance measure(s) to evaluate. Default is mean misclassification error (mmce)
* weights: [numeric] Optional, non-negative case weight vector to be used during fitting. If given, must be of same length as observations in task and in corresponding order. Overwrites weights specified in the task. By default NULL which means no weights are used unless specified in the task.
* models: [logical(1)] Should all fitted models be returned? Default is FALSE.
* keep.pred: [logical(1)] Keep the prediction data in the pred slot of the result object. If you do many experiments (on larger data sets) these objects might unnecessarily increase object size / mem usage, if you do not really need them. In this case you can set this argument to FALSE. Default is TRUE.
* show.info: [logical(1)] Print verbose output on console? Default is set via configureMlr.

Doc.: https://www.rdocumentation.org/packages/mlr/versions/2.10/topics/resample

In [66]:
r = resample(learner = learner
            ,task = task 
            ,resampling = resample 
            ,measures = measures
            #---------------------#
            ,models = TRUE
            ,keep.pred = FALSE
            ,show.info = TRUE
            )

Resampling: cross-validation
Measures:             mmce.test   acc.test    f1.test     ppv.test    tpr.test    auc.test    gini.test   
[Resample] iter 1:    0.2600000   0.7400000   0.4347826   0.5882353   0.3448276   0.8256435   0.6512870   
[Resample] iter 2:    0.2400000   0.7600000   0.5000000   0.7058824   0.3870968   0.8340346   0.6680692   
[Resample] iter 3:    0.2900000   0.7100000   0.4313725   0.6470588   0.3235294   0.7464349   0.4928699   
[Resample] iter 4:    0.2900000   0.7100000   0.5245902   0.6400000   0.4444444   0.7769097   0.5538194   
[Resample] iter 5:    0.2000000   0.8000000   0.5652174   0.8125000   0.4333333   0.8557143   0.7114286   
[Resample] iter 6:    0.2200000   0.7800000   0.4210526   0.5333333   0.3478261   0.8187465   0.6374929   
[Resample] iter 7:    0.1500000   0.8500000   0.6511628   0.7777778   0.5600000   0.8165333   0.6330667   
[Resample] iter 8:    0.2800000   0.7200000   0.3636364   0.6666667   0.2500000   0.8042279   0.6084559   
[Resampl

<h2>3. Result Analysis</h2>

Train Measures

In [67]:
r$measures.train

iter,mmce,acc,f1,ppv,tpr,auc,gini
1,0.02888889,0.9711111,0.9496124,1.0000000,0.9040590,0.9995013,0.9990027
2,0.03222222,0.9677778,0.9430255,1.0000000,0.8921933,0.9996230,0.9992459
3,0.02777778,0.9722222,0.9506903,1.0000000,0.9060150,0.9996146,0.9992291
4,0.03222222,0.9677778,0.9418838,1.0000000,0.8901515,0.9995414,0.9990828
5,0.02777778,0.9722222,0.9514563,1.0000000,0.9074074,0.9994121,0.9988242
6,0.03333333,0.9666667,0.9427481,1.0000000,0.8916968,0.9997161,0.9994321
7,0.03222222,0.9677778,0.9445507,0.9959677,0.8981818,0.9993309,0.9986618
8,0.03444444,0.9655556,0.9386139,1.0000000,0.8843284,0.9996517,0.9993033
9,0.02888889,0.9711111,0.9501916,1.0000000,0.9051095,0.9993004,0.9986008
10,0.03444444,0.9655556,0.9381238,1.0000000,0.8834586,0.9994604,0.9989208


Test Measures

In [68]:
r$measures.test

iter,mmce,acc,f1,ppv,tpr,auc,gini
1,0.26,0.74,0.4347826,0.5882353,0.3448276,0.8256435,0.6512870
2,0.24,0.76,0.5000000,0.7058824,0.3870968,0.8340346,0.6680692
3,0.29,0.71,0.4313725,0.6470588,0.3235294,0.7464349,0.4928699
4,0.29,0.71,0.5245902,0.6400000,0.4444444,0.7769097,0.5538194
5,0.20,0.80,0.5652174,0.8125000,0.4333333,0.8557143,0.7114286
6,0.22,0.78,0.4210526,0.5333333,0.3478261,0.8187465,0.6374929
7,0.15,0.85,0.6511628,0.7777778,0.5600000,0.8165333,0.6330667
8,0.28,0.72,0.3636364,0.6666667,0.2500000,0.8042279,0.6084559
9,0.24,0.76,0.4545455,0.5555556,0.3846154,0.7733888,0.5467775
10,0.33,0.67,0.2978723,0.5384615,0.2058824,0.7611408,0.5222816


Train Aggregated Result

In [69]:
apply(r$measures.train,2,mean)

iter       mmce        acc         f1        ppv        tpr        auc 
5.50000000 0.03122222 0.96877778 0.94508963 0.99959677 0.89626014 0.99951518 
      gini 
0.99903036

Test Aggregated Result

In [70]:
apply(r$measures.test,2,mean)

iter      mmce       acc        f1       ppv       tpr       auc      gini 
5.5000000 0.2500000 0.7500000 0.4644232 0.6465471 0.3681555 0.8012774 0.6025549

Run Time in seconds

In [71]:
r$runtime

[1] 6.64928

<h2>4. Prediction for new data</h2>

Read the data to predict

In [72]:
new.data = read.csv(file=sprintf('%s/%s', folder_name, file_name))

Search for the best model in crossvalidation and use it to score the incoming data

In [73]:
best.model = which.max(r$measures.test$acc)

In [74]:
pred = predict(r$models[[best.model]], newdata = new.data)

Prediction Result

In [75]:
pred

Prediction: 1000 observations
predict.type: prob
threshold: 1=0.50,2=0.50
time: 0.17
  truth    prob.1    prob.2 response
1     1 0.8592230 0.1407770        1
2     2 0.4648167 0.5351833        2
3     1 0.8917317 0.1082683        1
4     1 0.6769897 0.3230103        1
5     2 0.2949929 0.7050071        2
6     1 0.8007857 0.1992143        1
... (#rows: 1000, #cols: 4)

Cast result to data.frame to access the prediction

In [76]:
head(as.data.frame(pred))

truth,prob.1,prob.2,response
1,0.8592230,0.1407770,1
2,0.4648167,0.5351833,2
1,0.8917317,0.1082683,1
1,0.6769897,0.3230103,1
2,0.2949929,0.7050071,2
1,0.8007857,0.1992143,1
